# Running cellpose in colab with a GPU

(thanks to Matteo Carandini for setting this up)

UPDATED DEC 2020 for TORCH VERSION cellpose v0.6

## installation

Check CUDA version and GPU first

import libraries and check GPU (the first time you import cellpose the models will download).

In [3]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import scipy as sp
import tifffile
from cellpose import plot
import pandas as pd
import glob
%gui qt5

from urllib.parse import urlparse
from cellpose import models
import skimage.measure as measure
import skimage.transform as transform
import plotly.graph_objs as go

import cv2
import napari
import scipy.ndimage as ndimage
from cellpose import models
import glob
import plotly.express as px

In [4]:
imgs = []
labels = []

for f in glob.glob('./rDNA_train/*.tif'):
    labels.append(tifffile.imread(f)[-1])
    imgname = 'rgb/'+f.split('\\')[1]
    imgs.append(tifffile.imread(imgname))

In [5]:
cpm = models.CellposeModel(gpu=True, pretrained_model='S:/micro/jeg/tpo/smc/UBF_iFISH_HG02055_family/models/rDNA_model_v7', diam_mean=30)

svm = cpm.train(imgs, labels, test_data=None, test_labels=None,
          channels=[3,2], save_path='.', n_epochs=1000, rescale=True,
         normalize=True, batch_size=50)


100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 12.18it/s]


In [5]:
cpm.train?

In [1]:
model = models.Cellpose(gpu=True, model_type='cyto')
#model = models.Cellpose(gpu=True, model_type='bact_omni')
#model = models.CellposeModel(gpu=True, pretrained_model='c:/Users/catseanjohn/Dropbox/Work/TPO/models/acrocentric_model')


NameError: name 'models' is not defined

In [8]:
viewer = napari.Viewer()

In [6]:
model_chromosomes = models.CellposeModel(gpu=True, pretrained_model='models/rDNA_model_v7')

In [14]:
files = glob.glob('./rgb/*.tif')
idx = 32

In [15]:
current_file = files[idx]
img = tifffile.imread(current_file)
chromosomes = model_chromosomes.eval([img], channels=[3,2], diameter=30,
               normalize=True, resample=False)[0][0]

viewer.layers.clear()
viewer.add_image(img, blending='additive', channel_axis=2)
viewer.add_labels(chromosomes)
#viewer.add_points(np.array([x,y]).T)

<Labels layer 'chromosomes' at 0x1e384ea3670>

In [46]:
files[idx]

'./rgb\\HG002_1018.tif'

In [95]:
tifffile.imsave('./masks/'+current_file.split('\\')[1], viewer.layers[-1].data.astype(np.ushort), imagej=True)
idx = idx+1

In [66]:
def check_to_find(pts1, pts2):
    rslt = np.array([c==p for c in cpts for p in pts ])
    return np.sum(np.sum(rslt, axis=1)==2)==1

In [67]:
bads = []
for a in range(1,np.max(chromosomes)+1):
    nx, ny = np.where(chromosomes==a)
    cpts = np.array([nx, ny]).T
    found = check_to_find(pts, cpts)
    if found==False:
        bads.append(a)
bads = np.array(bads)

In [68]:
tmp = np.sum(np.array([chromosomes==b for b in bads]), axis=0)

In [69]:
viewer.add_image(tmp, blending='additive')

<Image layer 'tmp' at 0x1c51ca858c8>

In [49]:
rslt = np.array([c==p for c in cpts for p in pts ])

In [50]:
np.sum(np.sum(rslt, axis=1)==2)

1

In [24]:
idx

4

In [19]:
idx = idx+1

In [386]:
wwidth=65
wheight=130


In [388]:
def peak_finder(img, tol=300):
    x,y= findmaxima.findmaxima(img, tol)
    rtn = np.zeros(img.shape)
    rtn[x,y] = 255
    return rtn

In [389]:
def ptile(inp):
    return np.percentile(inp, 85)

In [390]:
def uniquer(inp):
    return len(np.unique(inp))

Download sample images

In [391]:
def generate_pt_list_edt(chromosomes, edt, medt, a):
    length = np.max(medt*(chromosomes==a).astype(int))
    pts = []
    for step in np.arange(0,1,0.2):
        cdis = np.round(step*length)
        tx, ty = np.unravel_index((edt*(np.round(medt)==cdis)).argmax(), medt.shape)
        pts.append([tx, ty])
    return pts

In [392]:
def generate_pt_list_cm(chromosomes, edt, medt, a):
    obj_mask = (chromosomes==a)
    length = np.max(medt*(obj_mask).astype(int))
    pts = []
    for step in np.arange(0,1.2,0.2):
        cdis = np.round(step*length)
        tx, ty = np.where((obj_mask*(np.round(medt)==cdis)))
        pts.append([np.mean(tx), np.mean(ty)])
    return pts

In [393]:
def get_closest_edge_point(chromosome_mask, medt, startx, starty):
    tmp = chromosome_mask.astype(np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    image = cv2.erode(tmp, kernel) 
    
    
    image = chromosome_mask.astype(np.uint8)-image
    tmp = medt*image.astype(int)
    tmp[tmp==0] = 1000
    minimum = np.min(tmp)
    xs, ys = np.where(tmp==minimum)
    return np.mean(xs), np.mean(ys)

In [394]:
def redistribute_points_uniformly(pts):
    diff = pts[1:] - pts[0:-1]
    dists = np.sqrt(diff[:,0]**2 + diff[:,1]**2)
    total_length = np.sum(dists)
    dists = np.concatenate([[0], dists])
    dists = np.cumsum(dists)
    xf = sp.interpolate.interp1d(dists, pts[:,0])
    yf = sp.interpolate.interp1d(dists, pts[:,1])
    rs = np.arange(0,total_length, np.round(total_length/5))
    nx = xf(rs)
    ny = yf(rs)
    return np.stack([nx, ny]).T

In [395]:
def get_angle_from_uniform_pts(pts):
    diff = pts[1:] - pts[0:-1]
    dists = np.sqrt(diff[:,0]**2 + diff[:,1]**2)
    total_length = np.sum(dists)
    dists = np.concatenate([[0], dists])
    dists = np.cumsum(dists)
    
    r = np.mean(diff[1:], axis=0)
    angle = np.arctan2(r[1], r[0])/3.1415926*180
    return angle

In [396]:
def pad_image_to(img, width, height):
    shp = img.shape
    newimg = img.copy()
    
    if (shp[-2]>width):
        offset = int(np.round((shp[-2]-width)/2))
        newimg = newimg[offset:(offset+width),:]
    if (shp[-1]>height):
        offset = int(np.round((shp[-1]-height)/2))
        newimg = newimg[:,offset:(offset+height)]

    shp = newimg.shape
    w = width-shp[-2]
    padl = int(np.round(w/2))
    padr = width-padl-shp[-2]
    h = height-shp[-1]
    padu = int(np.round(h/2))
    padd = height-padu-shp[-1]
    
    newimg = np.pad(newimg, ((padl, padr), (padu, padd)))
    return newimg

In [397]:
def crop_and_rotate(img, mask, angle):
    xs, ys = np.where(mask)
    bounds = np.array([[np.min(xs), np.min(ys)], [np.max(xs), np.max(ys)]])
    new_img = img[:,bounds[0,0]:bounds[1,0], bounds[0,1]:bounds[1,1]]
    new_mask = mask[bounds[0,0]:bounds[1,0], bounds[0,1]:bounds[1,1]]
    new_mask = pad_image_to(transform.rotate(new_mask, angle=-angle, resize=True), wheight, wwidth)
    
    
    tmp = new_mask.astype(np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    new_mask = cv2.dilate(tmp, kernel) 
    
    new_img = np.array([pad_image_to(transform.rotate(ab, angle=-angle, resize=True), wheight, wwidth)*new_mask for ab in new_img])
    return new_img, new_mask
    

In [368]:
img = tifffile.imread(glob.glob('./All/*.tif')[48])
DAPI = img[2]
chromosomes = model.eval([DAPI], channels=[0,0], diameter=60,
               normalize=True, resample=False)[0][0]
clear_layers()
viewer.add_image(img, channel_axis=0, blending='additive')
viewer.add_labels(chromosomes)


smoothed = sp.ndimage.gaussian_filter(DAPI, sigma=1.5)
edts = np.zeros(chromosomes.shape)
tips = np.zeros(chromosomes.shape)

start_list = []
stop_list = []

pt_list = []
nimgs = []
nmasks = []

props = measure.regionprops(chromosomes)
areas = np.array([p['Area'] for p in props])
sorted_chromosomes = np.argsort(-areas)+1

#for a in range(1, np.max(chromosomes)+1):
for a in sorted_chromosomes:
    bound = np.percentile(smoothed[chromosomes==a], 98)
    tmask = (smoothed>bound) & (chromosomes==a)
    tips[tmask] = a
    
    ttips = np.zeros(chromosomes.shape)
    ttips[tmask] = a
    labs = measure.label((ttips>0).astype(int))
    props = measure.regionprops(labs)
    startx, starty = props[0]['centroid']
    
    xs, ys = np.where(labs>0)
    startx = np.round(np.mean(xs))
    starty = np.round(np.mean(ys))
    start_list.append([startx, starty])
    
    
    
    to_transform = np.ones(chromosomes.shape)
    to_transform[int(startx), int(starty)] = 0
    #medt = cv2.distanceTransform((1-labs).astype(np.ubyte), cv2.DIST_L2,  maskSize = cv2.DIST_MASK_PRECISE)
    medt = cv2.distanceTransform(to_transform.astype(np.ubyte), cv2.DIST_L2,  maskSize = cv2.DIST_MASK_PRECISE)
    newx, newy = get_closest_edge_point(chromosomes==a, medt, startx, starty)
    endx, endy = np.unravel_index((medt*(chromosomes==a)).argmax(), medt.shape)
    stop_list.append([endx, endy])
    
    edt = cv2.distanceTransform((chromosomes==a).astype(np.ubyte), cv2.DIST_L2,  maskSize = cv2.DIST_MASK_PRECISE)
    edts = edts+edt
    
    current_pt_list = generate_pt_list_cm(chromosomes, edt, medt, a)
    nimg, nmask = crop_and_rotate(img, chromosomes==a, get_angle_from_uniform_pts(np.array(current_pt_list)))
    nimgs.append(nimg)
    nmasks.append(nmask)
    pt_list.append(current_pt_list)

viewer.add_labels(tips.astype(int))
viewer.add_image(edts, blending='additive')    
pt_list = np.array(pt_list)

In [369]:

columns = 12
rows = int(np.ceil(len(nimgs)/12))

master_img = np.zeros([3, wheight*rows, wwidth*columns])
idx = 0
for y in np.arange(0,wheight*rows, wheight):
    for x in np.arange(0,wwidth*columns, wwidth):
        master_img[:,y:(y+wheight), x:(x+wwidth)] = nimgs[idx]
        idx = idx+1
        if idx>=len(nimgs):
            break
clear_layers()
viewer.add_image(master_img, channel_axis=0)
viewer.layers[0].colormap='red'
viewer.layers[1].colormap='green'
viewer.layers[2].colormap='blue'
viewer.layers[1].contrast_limits=[60,600]

In [398]:
for f in glob.glob('./All/*.tif'):
    img = tifffile.imread(f)
    DAPI = img[2]
    chromosomes = model.eval([DAPI], channels=[0,0], diameter=60,
                   normalize=True, resample=False)[0][0]
    clear_layers()
    viewer.add_image(img, channel_axis=0, blending='additive')
    viewer.add_labels(chromosomes)


    smoothed = sp.ndimage.gaussian_filter(DAPI, sigma=1.5)
    edts = np.zeros(chromosomes.shape)
    tips = np.zeros(chromosomes.shape)

    start_list = []
    stop_list = []

    pt_list = []
    nimgs = []
    nmasks = []

    props = measure.regionprops(chromosomes)
    areas = np.array([p['Area'] for p in props])
    sorted_chromosomes = np.argsort(-areas)+1

    #for a in range(1, np.max(chromosomes)+1):
    for a in sorted_chromosomes:
        bound = np.percentile(smoothed[chromosomes==a], 98)
        tmask = (smoothed>bound) & (chromosomes==a)
        tips[tmask] = a

        ttips = np.zeros(chromosomes.shape)
        ttips[tmask] = a
        labs = measure.label((ttips>0).astype(int))
        props = measure.regionprops(labs)
        startx, starty = props[0]['centroid']

        xs, ys = np.where(labs>0)
        startx = np.round(np.mean(xs))
        starty = np.round(np.mean(ys))
        start_list.append([startx, starty])



        to_transform = np.ones(chromosomes.shape)
        to_transform[int(startx), int(starty)] = 0
        #medt = cv2.distanceTransform((1-labs).astype(np.ubyte), cv2.DIST_L2,  maskSize = cv2.DIST_MASK_PRECISE)
        medt = cv2.distanceTransform(to_transform.astype(np.ubyte), cv2.DIST_L2,  maskSize = cv2.DIST_MASK_PRECISE)
        newx, newy = get_closest_edge_point(chromosomes==a, medt, startx, starty)
        endx, endy = np.unravel_index((medt*(chromosomes==a)).argmax(), medt.shape)
        stop_list.append([endx, endy])

        edt = cv2.distanceTransform((chromosomes==a).astype(np.ubyte), cv2.DIST_L2,  maskSize = cv2.DIST_MASK_PRECISE)
        edts = edts+edt

        current_pt_list = generate_pt_list_cm(chromosomes, edt, medt, a)
        nimg, nmask = crop_and_rotate(img, chromosomes==a, get_angle_from_uniform_pts(np.array(current_pt_list)))
        nimgs.append(nimg)
        nmasks.append(nmask)
        pt_list.append(current_pt_list)

    viewer.add_labels(tips.astype(int))
    viewer.add_image(edts, blending='additive')    
    pt_list = np.array(pt_list)
    
    columns = 12
    rows = int(np.ceil(len(nimgs)/12))

    master_img = np.zeros([3, wheight*rows, wwidth*columns])
    idx = 0
    for y in np.arange(0,wheight*rows, wheight):
        for x in np.arange(0,wwidth*columns, wwidth):
            master_img[:,y:(y+wheight), x:(x+wwidth)] = nimgs[idx]
            idx = idx+1
            if idx>=len(nimgs):
                break
    tifffile.imsave('./Karyos/'+f.split('\\')[1], master_img.astype(np.single), imagej=True)

Mean of empty slice.
invalid value encountered in double_scalars
Mean of empty slice.
invalid value encountered in double_scalars
Mean of empty slice.
invalid value encountered in double_scalars
Mean of empty slice.
invalid value encountered in double_scalars
Mean of empty slice.
invalid value encountered in double_scalars


In [215]:
corrected_pts = [redistribute_points_uniformly(pts) for pts in pt_list]
corrected_pts = np.concatenate(corrected_pts)

In [199]:
viewer.add_points(corrected_pts.reshape([-1,2]), size=4)

<Points layer 'Points' at 0x1f1ed7d3bc8>

In [200]:
viewer.add_points(np.array(start_list))
viewer.add_points(np.array(stop_list))

<Points layer 'Points [2]' at 0x1f1ed86f348>

In [111]:
master = np.array([[100,100], [100,200], [100,300], [100, 400], [100,500]])

In [167]:
pt_list[0]

array([[145, 210],
       [134, 209],
       [122, 207],
       [111, 205],
       [100, 202]], dtype=int64)

In [173]:
h, status = cv2.findHomography(np.float32(pt_list[0]).reshape(-1,1,2), np.float32(master).reshape(-1,1,2))

In [174]:
h

In [135]:
test = np.swapaxes(img, 0,2).astype(float)

In [164]:
im_out = cv2.warpPerspective(img[2], h, (600,600))

In [165]:
im_out.shape

(600, 600)

In [166]:
viewer.add_image(im_out)

<Image layer 'im_out [2]' at 0x227d760a188>

In [147]:
pt_A = [41, 2001]
pt_B = [2438, 2986]
pt_C = [3266, 371]
pt_D = [1772, 136]

In [148]:
input_pts = np.float32([pt_A, pt_B, pt_C, pt_D])

In [149]:
input_pts.shape

(4, 2)

In [150]:
im_out = cv2.warpPerspective(input_pts, input_pts*2, (200,600))

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:3168: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 3 && M0.cols == 3 in function 'cv::warpPerspective'


In [152]:
M = cv2.getPerspectiveTransform(input_pts,input_pts*2)

In [56]:
sub = -edts[360:400, 300:340]
ptA = [33,14]
ptB=[11,35]

In [22]:
process_file(glob.glob('./All/*.tif')[16], display=True)

(                        File  Areas         IntDen  Chromosome  SeedSpots
 0   ./All\EPC_C2_022_MAX.tif   37.0   39393.535156         7.0        1.0
 1   ./All\EPC_C2_022_MAX.tif   37.0   39747.281250         7.0        1.0
 2   ./All\EPC_C2_022_MAX.tif   37.0   14571.961914        11.0        1.0
 3   ./All\EPC_C2_022_MAX.tif   58.0   36312.796875        12.0        1.0
 4   ./All\EPC_C2_022_MAX.tif   37.0   57336.195312        17.0        0.0
 5   ./All\EPC_C2_022_MAX.tif   37.0   49852.398438        27.0        1.0
 6   ./All\EPC_C2_022_MAX.tif   37.0   49953.988281        27.0        1.0
 7   ./All\EPC_C2_022_MAX.tif   37.0  104855.687500        33.0        1.0
 8   ./All\EPC_C2_022_MAX.tif   37.0  112719.085938        33.0        1.0
 9   ./All\EPC_C2_022_MAX.tif   74.0   97729.906250        35.0        2.0
 10  ./All\EPC_C2_022_MAX.tif   37.0   69155.898438        37.0        1.0
 11  ./All\EPC_C2_022_MAX.tif   37.0   15943.268555        38.0        1.0
 12  ./All\EPC_C2_022_MAX

In [41]:
df_UBF = pd.DataFrame()
df_rDNA = pd.DataFrame()
for f in glob.glob('./All/*.tif'):
    tmpU, tmpr = process_file(f)
    df_UBF = pd.concat([df_UBF,tmpU])
    df_rDNA = pd.concat([df_rDNA,tmpr])

In [42]:
df_UBF['Type'] = df_UBF['File'].str.split('\\').str[1].str.split('_').str[0]
df_rDNA['Type'] = df_rDNA['File'].str.split('\\').str[1].str.split('_').str[0]

In [43]:
df_UBF.to_csv('Results_UBF.csv')
df_rDNA.to_csv('Results_rDNA.csv')

# Analyze

### rDNA

In [293]:
df = pd.read_csv('Results_rDNA.csv')
df['Mean'] = df['IntDen']/df['Areas']

f=go.FigureWidget(
    px.box(df, y='Mean', x='Type', hover_data=['File'], points='all')
    )

                
def click_fn(trace, points, state):
    if (len(points.point_inds)>0):
        fname = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(fname)
        process_file(fname, display=True)
        
for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'custom…

In [294]:
fig = px.box(df, y='Mean', x='Type', hover_data=['File'], points='all')
fig.write_html("rDNA.html")

In [295]:
for a in df['Type'].unique():
    for b in df['Type'].unique():
        rslt = sp.stats.mannwhitneyu(df[df['Type']==a]['IntDen'], df[df['Type']==b]['IntDen'])
        print(a+','+b+': '+str(rslt[1]))

EPC,EPC: 0.4999584557824599
EPC,Naive: 6.781791631298397e-24
EPC,Neuron: 8.47872025743551e-22
Naive,EPC: 6.781791631298397e-24
Naive,Naive: 0.4999382698043157
Naive,Neuron: 0.4411815528481688
Neuron,EPC: 8.47872025743551e-22
Neuron,Naive: 0.4411815528481688
Neuron,Neuron: 0.4998107016796531


### UBF

In [296]:
df = pd.read_csv('Results_UBF.csv')
df['Mean'] = df['IntDen']/df['Areas']

f=go.FigureWidget(
    px.box(df, y='Mean', x='Type', hover_data=['File'], points='all')
    )

                
def click_fn(trace, points, state):
    if (len(points.point_inds)>0):
        fname = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(fname)
        process_file(fname, display=True)
        
for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'custom…

In [297]:
fig = px.box(df, y='Mean', x='Type', hover_data=['File'], points='all')
fig.write_html("UBF.html")

In [ ]:
for a in df['Type'].unique():
    for b in df['Type'].unique():
        rslt = sp.stats.mannwhitneyu(df[df['Type']==a]['IntDen'], df[df['Type']==b]['IntDen'])
        print(a+','+b+': '+str(rslt[1]))

### Ratio UBF/rDNA signal

In [139]:
df_UBF = pd.read_csv('Results_UBF.csv')
df_rDNA = pd.read_csv('Results_rDNA.csv')

df_UBF['Mean'] = df_UBF['IntDen']/df_UBF['Areas']
df_rDNA['Mean'] = df_rDNA['IntDen']/df_rDNA['Areas']

In [140]:
aggA = df_UBF.groupby(['Type', 'File']).agg({'Mean':np.median}).reset_index()
aggB = df_rDNA.groupby(['Type', 'File']).agg({'Mean':np.median}).reset_index()

agg = aggA.merge(aggB, on=['Type', 'File'])

In [141]:
agg['Ratio'] = agg['Mean_x']/agg['Mean_y']

In [142]:

f=go.FigureWidget(
    px.box(agg, x='Type', y='Ratio', points='all', hover_data=['File'])
    )

                
def click_fn(trace, points, state):
    if (len(points.point_inds)>0):
        fname = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(fname)
        process_file(fname, display=True)
        viewer.layers[0].colormap='green'
        for a in [0,1,2,3,4,5,6,7,8,9]:
            viewer.layers[a].visible=False
        for a in [0,2,9]:
            viewer.layers[a].visible=True
        
for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'custom…

./All\Naive_C2_047_MAX.tif


In [54]:
for a in agg['Type'].unique():
    for b in agg['Type'].unique():
        rslt = sp.stats.mannwhitneyu(agg[agg['Type']==a]['Ratio'], agg[agg['Type']==b]['Ratio'])
        print(a+','+b+': '+str(rslt[1]))

EPC,EPC: 0.4977528596899483
EPC,Naive: 8.318954067270528e-09
EPC,Neuron: 0.0006668950251175237
Naive,EPC: 8.318954067270528e-09
Naive,Naive: 0.4978429747012865
Naive,Neuron: 0.0005550743851390979
Neuron,EPC: 0.0006668950251175237
Neuron,Naive: 0.0005550743851390979
Neuron,Neuron: 0.49081796615144946


In [74]:
fig = px.box(agg, x='Type', y='Ratio', points='all', hover_data=['File'], title='Ratio of UBF to rDNA signal')
fig.write_html("ratio.html")

See how often the original seeds from channel 1 got merged in the channel 2 mask

In [62]:
agged = df.groupby(['File', 'Type']).agg({'SeedSpots':np.mean}).reset_index()

In [63]:
px.box(agged, x='Type', y='SeedSpots', points='all')

In [46]:
cdf = df.groupby(['File', 'Chromosome', 'Type']).agg({'IntDen':len}).reset_index()

In [47]:
agged = cdf.groupby(['File', 'Type']).agg({'IntDen':np.mean}).reset_index()

In [48]:
agged

,File,Type,IntDen
0,./All\EPC_C2_001_MAX.tif,EPC,1.400000
1,./All\EPC_C2_002_MAX.tif,EPC,1.000000
2,./All\EPC_C2_003_MAX.tif,EPC,1.750000
3,./All\EPC_C2_004_MAX.tif,EPC,1.600000
4,./All\EPC_C2_005_MAX.tif,EPC,1.230769
...,...,...,...
82,./All\Neuron_C2_024_MAX.tif,Neuron,1.875000
83,./All\Neuron_C2_025_MAX.tif,Neuron,14.000000
84,./All\Neuron_C2_026_MAX.tif,Neuron,1.625000
85,./All\Neuron_C2_027_MAX.tif,Neuron,1.875000


In [18]:
f=go.FigureWidget(
    px.box(agged, x='Type', y='IntDen', points='all', hover_data=['File'])
    )

                
def click_fn(trace, points, state):
    if (len(points.point_inds)>0):
        fname = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(fname)
        process_file(fname, display=True)
        
for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'custom…

./All\Neuron_C2_025_MAX.tif
./All\Neuron_C2_017_MAX.tif
./All\Neuron_C2_027_MAX.tif
./All\Naive_C2_001_MAX.tif
./All\Naive_C2_005_MAX.tif
./All\Naive_C2_016_MAX.tif
./All\Naive_C2_023_MAX.tif
./All\EPC_C2_002_MAX.tif
./All\Neuron_C2_026_MAX.tif
./All\EPC_C2_001_MAX.tif
./All\Neuron_C2_009_MAX.tif
./All\Neuron_C2_009_MAX.tif
